https://docs.python.org/3/reference/datamodel.html#implementing-descriptors
https://docs.python.org/3.7/howto/descriptor.html
http://sametmax.com/les-descripteurs-en-python/
https://rushter.com/blog/python-class-internals/

In [73]:
# Default behavior
class A(object):
    a = "A class attribute a"
    b = "A class attribute b"
    c = "A class attribute c"
    
class B(A):
    a = "B class attribute a"
    b = "B class attribute b"
    
    def __init__(self):
        self.a = "B instance attribute a"
        
b = B()
print(vars(A))
print(vars(B))
print(vars(b))
print(b.a)
print(b.b)
print(b.c)

{'__module__': '__main__', 'a': 'A class attribute a', 'b': 'A class attribute b', 'c': 'A class attribute c', '__dict__': <attribute '__dict__' of 'A' objects>, '__weakref__': <attribute '__weakref__' of 'A' objects>, '__doc__': None}
{'__module__': '__main__', 'a': 'B class attribute a', 'b': 'B class attribute b', '__init__': <function B.__init__ at 0x7f83bc2c1488>, '__doc__': None}
{'a': 'B instance attribute a'}
B instance attribute a
B class attribute b
A class attribute c


In [65]:
class A(object):
    a_class_attr = 1
    
    def __init__(self, arg):
        self.a_inst_attr = arg
    
    def some_method(self):
        pass
    
class B(A):
    """
    lol
    """
    b_class_attr = 2
    
    def __init__(self, arg):
        self.b_inst_attr = arg
    
    def some_other_method(self):
        print(b_class_attr)
    
a = A('a')
b = B('b')
print(vars(A).keys())
print(A.__dict__['__dict__'])
print(vars(a))
print(vars(B).keys())
print(vars(b))
B.__dict__

dict_keys(['__module__', 'a_class_attr', '__init__', 'some_method', '__dict__', '__weakref__', '__doc__'])
<attribute '__dict__' of 'A' objects>
{'a_inst_attr': 'a'}
dict_keys(['__module__', '__doc__', 'b_class_attr', '__init__', 'some_other_method'])
{'b_inst_attr': 'b'}


mappingproxy({'__module__': '__main__',
              '__doc__': '\n    lol\n    ',
              'b_class_attr': 2,
              '__init__': <function __main__.B.__init__(self, arg)>,
              'some_other_method': <function __main__.B.some_other_method(self)>})

In [60]:
b.__getattribute__

NameError: name 'b_class_attr' is not defined

In [12]:
class A(object):
    def some_method(self):
        pass
    
a = A()

In [9]:
A.some_method

<unbound method A.some_method>

In [10]:
type(A.some_method)

instancemethod

In [5]:
a.some_method

<bound method A.some_method of <__main__.A object at 0x7ff78aa86d50>>

In [8]:
type(a.some_method)

instancemethod

In [2]:
a.__dict__

{}

In [5]:
def f():
    pass

In [14]:
property(fget=None, fset=None, fdel=None, doc=None)

In [11]:
staticmethod(None)

In [12]:
# classic look-up without descriptors
class A(object):
    a = 'class attr a'
    
class B(A):
    def __init__(self):
        self.a = 'instance attr a'
        
b = B()
print('-------')
print (b.a)
print('-------')
print (B.a)

-------
instance attr a
-------
class attr a


In [1]:
class DataDescriptor(object):
    def __init__(self, initval=None, name='var'):
        self.val = initval
        self.name = name

    def __get__(self, obj, objtype):
        print('Retrieving', self.name)
        out = self.val
        print(out)
        return out

    def __set__(self, obj, val):
        print('Updating', self.name)
        print(val)
        self.val = val
        
class NonDataDescriptor(object):
    def __init__(self, initval=None, name='var'):
        self.val = initval
        self.name = name

    def __get__(self, obj, objtype):
        print('Retrieving', self.name)
        return self.val

In [107]:
# look-up with data descriptors
class O(object):
    d = NonDataDescriptor(initval='value stored in O NonDataDescriptor', name='d')
    r = DataDescriptor(initval='value stored in O DataDescriptor', name='r')

class A(O):
    a = 'class attr a'
    d = DataDescriptor(initval='value stored in A DataDescriptor', name='d')
    r = NonDataDescriptor(initval='value stored in A NonDataDescriptor', name='r')
    
class B(A):
    def __init__(self):
        self.a = 'instance attr a'
#         self.d = 'instance attr d' # Ne sert à rien, self.d va chercher le descripteur et appelle __set__
        self.r = 'instance attr r' # Descripteur surchargeable car ne définissant pas de __set__ ?
        
b = B()
print(vars(b))
b.__dict__['d'] = 'instance attr d' # la seule façon de tester la priorité des data descriptors sur les variables d'instance
print(vars(b))
print(b.d)
print(B.d)
print(b.r)
print(B.r)

{'a': 'instance attr a', 'r': 'instance attr r'}
{'a': 'instance attr a', 'r': 'instance attr r', 'd': 'instance attr d'}
Retrieving d
value stored in A DataDescriptor
value stored in A DataDescriptor
Retrieving d
value stored in A DataDescriptor
value stored in A DataDescriptor
instance attr r


In [93]:
B.d

Retrieving d
value stored in A DataDescriptor


'value stored in A DataDescriptor'

In [78]:
A.__dict__['d'].val

'instance attr d'

In [75]:
A.__dict__['d'].__get__(None, B)

Retrieving d
instance attr d


'instance attr d'

In [16]:
class NonDataDescriptor(object):
    def __init__(self, value):
        self.value = value
        
    def __get__(self, obj, owner):
        print("Retrieving value")
        return self.value
    

In [17]:
# look-up with non-data descriptor
class A(object):
    a = 'class attr a'
    d = NonDataDescriptor('value stored in DataDescriptor d')
    
class B(A):
    def __init__(self):
        self.a = 'instance attr a'
        self.d = 'instance attr d'
        
b = B()
print('-------')
print (b.d)
print('-------')
print (B.d)

-------
instance attr d
-------
Retrieving value
value stored in DataDescriptor d


In [21]:
# look-up with non-data descriptor
class O(object):
    d = 'class attr d'

class A(O):
    a = 'class attr a'
    d = NonDataDescriptor('value stored in DataDescriptor d')
    
class B(A):
    def __init__(self):
        self.a = 'instance attr a'
        self.d = 'instance attr d'
        
b = B()
print('-------')
print (b.d)
print('-------')
print (B.d)

-------
instance attr d
-------
Retrieving value
value stored in DataDescriptor d


In [22]:
# look-up with non-data descriptor
class O(object):
    d = DataDescriptor('value stored in DataDescriptor d')

class A(O):
    a = 'class attr a'
    d = 'class attr d'
    
class B(A):
    def __init__(self):
        self.a = 'instance attr a'
        self.d = 'instance attr d'
        
b = B()
print('-------')
print (b.d)
print('-------')
print (B.d)

-------
instance attr d
-------
class attr d


In [31]:
# look-up with non-data descriptor
class O(object):
    d = DataDescriptor('value stored in DataDescriptor d')

class A(O):
    a = 'class attr a'
    d = 'class attr d'
    
    @classmethod
    def some_cls_method(cls):
        print(cls.__module__)
    
class B(A):
    def __init__(self):
        self.a = 'instance attr a'

        
b = B()
print('-------')
print (b.d)
print('-------')
print (B.d)

-------
class attr d
-------
class attr d


In [34]:
type(b.some_cls_method)

method

In [28]:
dir(b)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'a',
 'd',
 'some_cls_method']

In [3]:
a = 'global'
class A(object):
    a = 'class attr'
    # Du point de vue du nom a ci-dessous, c'est la règle LEGB qui s'applique, le scope
    # local étant le corps de la classe
    b = a 
    def __init__(self):
        self.a = 'inst attr'
    # L'enclosing scope d'une méthode est l'enclosing scope de la classe 
    # et non le reste du corps de la classe
    def do_smth(self):
        print(a)
    
class B(object):
    b = a
    
a_inst = A()
a_inst.do_smth()
print(a_inst.b)
b_inst = B()
print(b_inst.b)

global
class attr
global


In [50]:
type(A.do_smth)

function

In [29]:
class C(object):
    def some_instance_method(self):
        pass
    
    @classmethod
    def some_class_method(cls):
        pass
    
    @staticmethod
    def some_static_method():
        pass
    
c = C()

In [30]:
print(C.__dict__['some_instance_method'])
print(C.__dict__['some_class_method'])
print(C.__dict__['some_static_method'])

<function C.some_instance_method at 0x7fe95c7830d0>


In [31]:
print(type(C.__dict__['some_instance_method']).__name__)
print(type(C.__dict__['some_class_method']).__name__)
print(type(C.__dict__['some_static_method']).__name__)
# Pas étonnant 
# 1. Une méthode d'instance est définie comme une fonction et en fait stockée comme tel dans
# le __dict__
# 2. Le décorateur se traduit par le passage de la méthode d'instance "classique" au 
# constructeur du descripteur Classmethod qui est l'objet finalement stocké dans __dict__
# 3. Idem que 2. mais pour le constructeur du descripteur Staticmethod

function
classmethod
staticmethod


In [33]:
print(C.some_instance_method)
print(C.some_class_method)
print(C.some_static_method)

<function C.some_instance_method at 0x7fe95c7830d0>
<bound method C.some_class_method of <class '__main__.C'>>
<function C.some_static_method at 0x7fe95c783268>


In [9]:
print(type(C.some_instance_method).__name__)
print(type(C.some_class_method).__name__)
print(type(C.some_static_method).__name__)

function
method
function


In [34]:
print(c.some_instance_method)
print(c.some_class_method)
print(c.some_static_method)

<bound method C.some_instance_method of <__main__.C object at 0x7fe95c799048>>
<bound method C.some_class_method of <class '__main__.C'>>
<function C.some_static_method at 0x7fe95c783268>


In [12]:
print(type(c.some_instance_method).__name__)
print(type(c.some_class_method).__name__)
print(type(c.some_static_method).__name__)

method
method
function


### Les dictionnaires de classe et d'instance 
Une classe (*base classes* comme les métaclasses) comme les instances de classes constituent chacune du point de vue du *lexical scoping* un environnement local. Ce namespace est dans les deux cas implémenté à l'aide d'un dictionnaire dans lequel sont recherchées les références à un attribut. Ce dictionnaire est accessible et correspond à l'attribut ```__dict__```. L'appel à la *built-in function* ```vars``` retourne également ```__dict__```.

Remarque : la notion d'environnement local est plus évidente pour les fonctions, cet environnement étant borné au corps de la fonction (*lexical scoping*). Le *namespace* correspondant est accessible via l'attribut ```__dict__``` de la fonction. Cela n'a rien d'étonnant, les fonctions étant en Python des objets à part entières, des instances du type ```function``` (*functions as first-class citizen*). De façon analogue dans le cas des classes, l'étendue de l'environnement local correspond au corps de la classe.

Le dictionnaire d'une classe contiendra attributs de classes et méthodes (qui dans le dictionnaire sont des objets ```functions```). Le dictionnaire d'une instance contiendra les attributs d'instance (le plus souvent définis dans ```__init__```).

Remarque : Les updates du ```__dict__``` ne se font pas directement mais via les *magic methods* ```__setattr__``` et ```__delattr__``` ou leurs *built-ins* correspondant : ```setattr``` et ```del``` respectivement.

Attention : En présence de descripteurs (cf. plus loin), l'objet retourné lors de l'*attribute access* peut être différent de celui présent dans le ```__dict__``` : ```C.__dict__['x']``` et ```C.x``` peuvent être différents. Par exemple, pour une *static method*, l'objet stocké dans le dictionnaire est un descripteur de type ```staticmethod``` mais celui retourné via l'*attribute access* est une ```function```.

#### Différence entre les *built-ins* ```dir``` et ```vars```
```vars``` retourne le contenu du ```__dict__``` de l'objet qui lui est passé. Si l'objet est un ```object```, ```vars``` retourne un ```dict```. Dans le cas d'un ```type```, ```vars``` retourne un object ```MappingProxy```. Dans les deux cas, il s'agit d'un objet implémentant la classe abstraite ```Mapping```. ```dir``` retourne une liste d'attributs valides pour l'objet qui lui est passé (sous forme d'une ```list``` de ```str```). Le contenu est fonction de l'implémentation faite de la *magic method* ```__dir__``` mais dans le cas général, il s'agira du contenu du ```__dict__``` de l'object, de celui de sa *base class*, etc. jusqu'à en arriver à ```object``` (si l'objet passé est une instance ou une *base class* héritant ultimement de ```object```) ou ```type``` (si l'objet passé est une métaclasse). ```dir``` récolte en fait récursivement les noms de tous les objets accessibles via l'attribute access. 

Exemple rappelant que les méthodes sont des attributs du type et non de l'instance.

### *Attribute access* : *la magic method* ```.```
L'access à un attribut passe le plus souvent par l'usage de l'*infix* ```.``` (plus "naturel" que l'utilisation équivalente de la built-in function ```getattr```), sucre syntaxique équivalent à l'appel de la *class method* ```__getattribute__```. Sans surcharge, c'est finalement la méthode ```__getattribute__``` de ```object``` ou de ```type``` qui est appelée en fonction du type d'*attribute access*:
* Le *class attribute access*: ```A.x``` est équivalent à ```type.__getattribute__(A, 'x')``` : on requête l'attribut ```x``` de l'instance ```A``` du type ```type```.
* L'*instance attribute access*: ```a.x``` est équivalent à ```object.__getattribute__(a, 'x')``` : on requête l'attribut ```x``` de l'instance ```a``` (de la classe ```A```) du type ```object```.

Remarque : On parle aussi de *binding* (liaison) et plus précisément de *class* et *instance binding* pour ```b.x``` et ```B.x``` respectivement - cf. plus loin.

Qu'il soit de type ```object``` ou ```type```, tout objet implémente les méthodes ```__getattribute__``` (à distinguer de ```__getattr__```, cf. plus loin), ```__setattr__```, et ```__delattr__``` qui sont au minimum héritées de ```object``` ou ```type```. La première implémente l'accès à l'attribut d'un objet, la seconde l'assignation d'une valeur à un attribut (opérations ```a.x = val``` ou ```A.x = val```) et la dernière supprimant la paire clé-valeur du dictionnaire de l'objet. Ces trois *magic methods* sont également celles appelées lors de l'utilisation des *built-in functions* ```getattr```, ```setattr``` et du *statement* ```del``` respectivement. 

Remarque: Ne pas confondre les méthodes précitées avec ```__getitem__``` et ```__setitem__``` qui sont les *magics methods* derrière le sucre syntaxique ```x[]``` permettant l'accès à une valeur via son index. Ex: ```some_list[0]```, ```some_dict['some_key']```.

L'*attribute access* fait exception à la règle LEGB qui ne s'appliquent pas à la résolution du nom de l'attribut recherché (en fait seul le L s'applique : on commence par aller voir dans le ```__dict__``` de l'objet qui correspond au *namespace local*). D'autres règles s'appliquent notamment dans les cas où:
* Le nom de l'attribut n'a pas été trouvé dans le ```__dict__``` de l'objet (ie: dans l'environnement local) sur lequel l'*attribute access* a été appelé. Où alors remonter comme on le ferait dans la règle LEGB ?
* Le nom de l'attribut recherché se trouve à plusieurs endroits de la *look-up chain*. Quelle règle de précédence alors appliquer ? On verra que la présence de descripteurs vient modifier la précédence la plus "naturelle" du premier rencontré, premier retourné dans le cas de l'*instance attribute access*.

### Les descripteurs
Un descripteur est un objet dont la classe implémente certaines méthodes bien définies (on parle aussi de *Descriptor protocol*). Ces objets ont de particulier qu'ils ne peuvent être qu'attributs de classe, la classe dont ils sont attribut étant appelée ***owner class***. Le descripteur peut en particulier faire office de *getter* (et éventuellement de *setter*) pour cet attribut. Exemples d'application : dans le cas d'une API : on peut modifier les *getters*/*setters* sans casser l'API, un *setter* peut permettre d'implémenter une logique de validation des données, des gains de performance en rendant l'évaluation des attributs lazy, etc.. Les applications des descripteurs sont assez larges et permettent par exemple une implémentation élégante de l'encapsulation ou de certains *design patterns* en Python (cf. l'exemple d'un *observer* dans l'article de [Sam & Max](http://sametmax.com/les-descripteurs-en-python/). 

Les descripteurs sont également à la base de l'implémentation d'importantes *features* de Python, les classes ```property```, ```classmethod```, ```staticmethod``` (qui se cachent derrière les décorateurs ```@property```, ```@classmethod``` et ```@staticmethod``` respectivement) et ```function``` implémentant toutes le *descriptor protocol*. L'implémentation du *built-in* ```super``` se repose également sur le *descriptor protocol*. 

Techniquement, un descripteur Python est une classe implémentant une ou plusieurs des trois méthodes ```__get__```, ```__set__``` et ```__del__```. On distingue notamment les cas particuliers : 
* D'une classe implémentant les méthodes ```__get__``` **et** ```__set__``` : on parle de ***data descriptor***.
* D'une classe implémentant la méthode ```__get__``` **mais pas** ```__set__``` : on parle de ***non-data descriptor***.

Dans le contexte de l'*attribute access*, la présence de descripteurs importe car elle peut modifier l'ordre dans lequel un attribut est recherché et donc éventuellement la valeur retournée. En particulier, l'accès à un attribut qui est également un descripteur va retourner la valeur produite par ```__get__``` et non l'objet lui-même.

On rappelle qu'un descripteur ne peut être qu'un attribut de classe (membre du ```__dict__``` de la classe). Un descripteur attribut d'instance n'a pas plus de sens que de contrôler l'accès aux attributs d'une instance en particulier. Par ailleurs, l'implémentation de ```__getattribute__``` n'attend les descripteurs que comme attributs de classe : si on assigne un descripteur à un attribut d'instance, alors ```a.x``` ne retournera pas la valeur produite par ```__get__``` mais l'objet descripteur lui-même.

### *Look-up chain* et *look-up order* en l'absence de descripteurs
Cette partie vise à expliquer quelle est la chaîne d'environnement utilisée pour la résolution du nom ```x``` lors de l'évaluation de ```a.x``` ou de ```A.x``` (*look-up chain*) et dans quel est l'ordre de priorité prévalant si le nom est présent à plusieurs endroits de la chaîne (*look-up order*).
Que des attributs de classe soient des descripteurs ou pas, la *look-up chain* correspondra toujours : 
* Dans le cas de l'*instance attribute access* : au dictionnaire de l'instance suivi des dictionnaires des différentes *base classes* dans l'ordre MRO.
* Dans le cas du *class attribute access* : au dictionnaire de la classe suivi des dictionnaires de ses différentes *base classes* dans l'ordre MRO.

Dans les deux cas, les dictionnaires de la ou des métaclasse(s) ne font pas partie de la *look-up chain*. De même, dans les deux cas, si le nom recherché n'a pu être trouvé dans aucun des dictionnaires de la look-up chain, la méthode ```__getattr__``` est finalement appelée retournant au pire une ```AttributeError```. Si ```__getattr__``` n'est pas implémentée, on se voit de la même façon retourner une ```AttributeError```.

Remarque : Dans le cas d'un *class attribute access* ```A.x``` où ```A``` est une métaclasse, la même règle s'applique que pour les *base classes* : la *look-up chain* est constituée du dictionnaire de la métaclasse et de ceux de ses parents dans l'ordre MRO.

**En l'absence de descripteurs, la valeur retournée se trouve toujours dans le premier dictionnaire de la *look-up chain* présentant la clé (le nom) recherché.**

Remarque : l'invocation implicite des *special methods* (invocation explicite: ```x.__len__()``` vs invocation implicite : ```len(x)```) fait exception à la procédure écrite ci-dessus : 
* ```__getattribute__``` est contournée et n'est pas appelée : ```x.__len__()``` entraine un appel à ```object.__getattribute__``` mais pas ```len(x)```.
* Le dictionnaire de classe ne fait pas partie de la look-up chain, sans celà certaines *magic methods* également implémentées pour les ```type``` comme ```hash``` ou ```repr``` ne seraient jamais appelées : ```repr(int)``` n'appelerait jamais ```type.__repr__()```.  

#### Différence entre ```__getattribute__``` et ```__getattr__``` :
```__getattr__``` n'est appelée que si l'attribut recherché n'a pas pu être trouvé suivant la procédure normale de recherche. L'implémenter est intéressant si on souhaite avoir un comportement particulier en cas de requête d'un attribut non encore existant (retour d'une valeur par défaut, etc.). ```__getattribute__``` est à l'opposé toujours appelée lors de la requête d'un attribut et c'est précisément cette méthode qui implémente la procédure de *look-up* décrite ci-dessus. C'est ```__getattribute__``` qui peut être amenée à appeler ```__getattr__``` en dernière nécessité.

Remarque : afin d'être appelée, la méthode ```__getattr__``` doit respecter la signature suivante : ```__getattr__(self, attr)```.

Si la *looked-up value* se révèle être un objet descripteur, alors le *look-up order* "traditionnel" présenté ci-dessus peut être chamboulé, les différences dépendant de la nature du descripteur (*data descriptor* ou *non-data descriptor*) et de comment il a été appelé (*class* ou *instance attribute access*).

### *Look-up order* en présence de descripteurs dans la *look-up chain*
L'impact de la présence d'un descripteur sur la valeur retourné lors de l*attribute access* dépend de sa nature ainsi que de celle de l'objet (```object``` ou ```type```) sur lequel a été appelé le *dotted access*. 

En présence de descripteurs dans les attributs de classe, le *look-up order s'organise comme suit: 
* *Class attribute access* : La valeur retournée est prise dans le premier dictionnaire du *look-up order* où apparait le nom. Soit est retourné l'objet correspondant à la valeur (inclut le cas d'un descripteur sans ```__get__```), soit la valeur retournée par ```__get__``` si l'objet associé à la clé se révèle être un descripteur implémentant la méthode. Si la clé n'apparait dans aucun dictionnaire des classes du MRO alors ```__getattr__``` est finalement appelée retournant au pire une ```AttributeError```. Il n'y a finalement quasiment pas de différence avec le cas de l'absence de descripteurs mis à part l'appel à ```__get__``` au lieu de retourner l'objet.
* *Instance attribute access* : Si le nom existe à la fois dans le dictionnaire de l'instance et dans au moins un des dictionnaires des *bases classes*, alors la valeur retournée est déterminée par l'ordre de priorité suivant (arbitrage entre la valeur du dictionnaire d'instance et la valeur présente dans le premier dictionnaire de classe du MRO contenant le nom recherchée): 
    * Un *data descriptor* (donc forcément dans le dictionnaire d'une *base class*) a priorité sur une variable d'instance.
    * Une variable d'instance a priorité sur un *non-data descriptor* ou tout autre variable de classe qui n'est pas un descripteur.
    * Un *non-data descriptor* (ou tout autre variable de classe) a priorité sur ```__getattr__``` (appelé si le *look-up* du nom n'a rien donné).

Les procédures de *look-up* décrites ci-dessus sont implémentées par les méthodes ```__getattribute__``` de ```object``` et de ```type``` pour l'*instance attribute access* et le *class arribute access respectivement*. L'appel d'un descripteur destiné à retourner la valeur d'un attribut ```x``` correspond finalement au code suivant : 
* Dans le cas de l'*instance attribute access* : ```object.__getattribute__``` finit par exécuter ```type(c).__dict__['x'].__get__(c, type(c))``` (si *binding* il doit y avoir dans ```__get__```, c'est l'instance qui sera liée à l'objet retourné par ```__get__```).
* Dans le cas du *class attribute access* : ```type.__getattribute__``` finit par exécuter ```C.__dict__['x'].__get__(None, C)``` (si binding il doit y avoir dans ```__get__```, c'est le type, la classe qui sera liée à l'objet retourné par ```__get__```).

Remarque: Surcharger ```__getattribute__``` est déconseillé car on risque fortement de ne pas réussir à conserver la logique présentée ci-dessus, les descripteurs risquant par exemple de ne plus être appelés.

Remarque : **Rapports entre *attribute look-up order* et règle LEGB** : Il doit sans doute falloir ici distinguer variable liée et variable libre (définit comme le cas opposé) : 
* Quand on écrit ```A.x```, la règle LEGB est en quelque sorte ignorée : la chaîne de *namespaces* servant à la résolution du nom ```x``` est donnée par l'implémentation de la méthode ```__getattribute__``` de la classe ```A``` à laquelle ```x``` est lié. On est dans le cas dit **lié** (*bound*), le nom ```x``` est lié à un contexte particulier permettant sa résolution : l'objet ```A```. 
* A l'opposé on est dans le cas dit **libre** : si on écrit simplement ```x``` (même dans le corps d'une classe ou d'une méthode), c'est la règle LEGB qui va s'appliquer pour la résolution des noms (le corps de la classe ou de la méthode faisant office d'environnement local). On rappelle au passage que dans le cas des fonctions, une *closure* se définit comme le couple formé par la fonction et l'ensemble de ses variables libres.

### Un exemple d'utilisation des descripteurs pour le contrôle de l'accès : l'objet ```property```
Remarque : ```property(fget, fset, fdel, doc)``` retourne un objet (un *data descriptor*) de type ```property```.

Le code à exécuter autour de l'accès, l'assignation ou la suppression est centralisé au niveau de la classe dans le descripteur, il est ainsi le même pour toutes les instances.  

```property``` possède des méthodes de classe (```property.getter```, ```property.setter```, ```property.deleter```) qui permettent aussi de décorer plus finement des méthodes. 

https://docs.python.org/3.7/howto/descriptor.html

### *Binding behavior* : exemple des méthodes 
Grace à leur méthode ```__get__```, les descripteurs peuvent posséder un *binding behavior* : ils peuvent retourner un objet liant plusieurs objets dont celui ayant suscité l'*attribute access*. C'est la logique derrière l'implémentation des *class methods* en Python par exemple. Pour une fonction définie dans le corps d'une classe et décorée avec ```@classmethod```, c'est un objet ```classmethod``` qu'on trouvera dans le dictionnaire de la classe. Cet objet est également un descripteur. Quand un objet cherchera à avoir accès à la méthode, ```__get__``` ne va pas retourner une ```function``` mais une ```method``` (*bound method*): si on souhaite l'appeler, pas besoin de lui passer le type explicitement, l'objet ```method``` lie justement le type et la fonction, l'appel se fera implicitement sur ce type. 

L'objet ```staticmethod``` fonctionne de manière analogue mais retourne logiqument un objet ```function``` (*unbound method*) quel que soit le type d'accès (peut paraître ne servir à rien mais cela permet réellement de se distinguer d'une méthode d'instance qui retourne une *bound method* en *instance attribute access*). On remarque qu'il n'existe pas de décorateur ```@instancemethod``` : c'est par ce que les fonctions (objets ```function```) sont déjà des descripteurs. L'objet function possède en effet une méthode ```__get__``` qui suivant les arguments qui lui sont passés va retourner un object ```function``` ou ```method``` (*bound method*). Les arguments lui sont notamment passés par ```__getattribute__``` en fonction du type d'accès. 

Le code donne une idée de ce que pourrait donner en pur Python les comportements décrits ci-dessus: 
```python
class Function(object):
    
    def __get__(self, obj, type=None):
        if obj is None:
            return self
        return types.MethodType(self, obj)

class StaticMethod(object):

    def __init__(self, f):
        self.f = f

    def __get__(self, obj, type=None):
        return self.f
    
class ClassMethod(object):

    def __init__(self, f):
        self.f = f

    def __get__(self, obj, type=None):
        if type is None:
            type = obj.__class__
        return types.MethodType(self.f, type)
```

Remarque: comme pour ```super``` ou ```property```, l'appel à ```staticmethod``` et ```classmethod``` ne sont en fait pas des appels à des fonctions mais des appels au constructeur des classes correspondantes.

|*method* / *access type*|*instance access*|*class access*|*dict access*|
|---|---|---|---|
|*__instance method__*|*bound* ```method```|```function```|```function```|
|*__class method__*   |*bound* ```method```|*bound* ```method```|```classmethod```|
|*__static method__*  |```function```|```function```|```staticmethod```|

Les arguments de la méthode ```__get__``` des descripteurs servent à passer l'objet à éventuellement lier dans la valeur de retour. Les signatures des trois méthodes caractérisant un descripteur sont les suivantes : 
* ```__get__(self, obj, type=None)``` : Méthode utilisée pour récupérer la valeur de l'attribut de l'instance ```obj``` (*instance attribute access*) ou de la classe ```type``` (*class attribute access*). Cette méthode retourne une valeur.
* ```__set__(self, obj, value)``` : Méthode utilisée pour assigner une nouvelle valeur ```value``` à l'attribut de l'instance ```obj```. Cette méthode n'a pas de valeur de retour.
* ```__delete__(self, obj)``` : Méthode utilisée pour supprimer l'attribut de l'instance ```obj```. Cette méthode n'a pas de valeur de retour.

Dans le cadre du *look-up* implémenté par ```__getattribute__```, Les valeurs passées à ces fonctions dépendent du type d'accès. Ainsi, pour l'accès à un attribut ```x``` correspondant à un descripteur, ```__getattribute__``` transforme : 
* ```a.x``` en ```type(a).__dict__['x'].__get__(a, type(a))```
* ```A.x``` en ```A.__dict__['x'].__get__(None, A)```

En particulier, l'argument ```type``` de ```__get__``` est là pour le *class attribute access* (on trouve parfois cet argument nommé ```owner```, c'est trompeur, le type passé n'étant pas toujours l'*owner class* mais pouvant aussi être une sous-classe). A l'intérieur de ```__get__``` ces arguments permettent de faire la distinction entre les deux types d'accès (```if obj is None```) et de passer à ```__get__``` l'objet qu'il doit éventuellement se retrouver lié dans la valeur de retour.

### L'objet ```super```
Commençons par préciser que le built-in ```super``` n'est pas une fonction mais un objet : l'appel à ```super()``` est en fait un appel au constructeur de la classe ```super``` et retourne une instance de la classe ```super```.

```super``` est un objet auquel on délègue le *look-up* d'attributs, le plus souvent des méthodes mais cela fonctionne quelque soit la nature du membre à récupérer (on peut récupérer de simples attributs). Ce faisant, on se repose en fait sur la méthode ```__getattribute__``` de ```super``` plutôt que sur celle d'```object``` ou de ```type``` pour l'*attribute access*. 

Remarque: Dans ```super(C, c).some_method()```, il se passe en fait trois choses:
```python
super(C, c).some_method() # est équivalent à : 

super_obj = super(C, c) # 1. instanciation d'un objet super
mthd = super_obj.some_method # 2. récupération de la méthode via le dotted access / __getattribute__ de super
mthd() # 3. appel de la méthode
```

Le constructeur de ```super``` prend deux arguments: 
* Le premier est toujours un type, il s'agit du point de départ de la *look-up chain*, cette dernière correspondant au MRO de l'objet passé en second argument. L'objet passé à ce premier argument est assigné à l'attribut ```__thisclass__``` de l'objet ```super``` retourné. 
* Le second attribut peut être un ```object``` ou un ```type``` avec comme restriction d'être une instance ou une sous-classe du premier argument respectivement. C'est le MRO de cet objet qui va constituer la *look-up chain*. L'objet passé en second argument est égalment celui auquel sera lié (*bound*) l'attribut recherché. Dans le cas des méthodes, cela va influer sur le type d'objet retourné (*bound* ou *unbound*): cf. Binding behavior. Par exemple ```super(C, c).some_method()``` peut être vu comme équivalent à ```c.some_method()``` (où la liaison entre ```c``` et ```some_method``` est explicite) mais avec un *look-up* réalisé avec le ```__getattribute__``` de ```super``` et non d'```object```. L'objet passé en second argument est assigné à l'attribut ```__self__``` de l'objet ```super``` retourné. Il existe également un attribut ```__self_class__``` égal au type du second argument ou à ```__self__``` dans les cas où ce second argument est un ```object``` ou un ```type``` respectivement. Le second argument est en réalité optionnel. En absence de second argument l'objet ```super``` est dit *unbound* et les attributs ```__self__``` et ```__self_class__``` sont égaux à ```None```.

Remarque: Python 3 autorise un appel au constructeur de ```super``` sans arguments qui est équivalent à ```super(C, c)``` où ```c``` est l'instance courante dans le corps de la classe de laquelle est écrit le ```super()```.

Les règles de *look-up* décrites plus haut s'appliquent de la même façon pour ```super``` notamment pour ce qui a trait à la présence de descripteurs. La seule chose qui diffère avec le ```__getattribute__``` classique de ```object``` ou ```type``` est la *look-up chain* utilisée sur laquelle on a un contrôle direct via le premier argument du constructeur de ```super```. Il semble également que ```super``` ne déclenche pas d'appel à ```__getattr__``` en cas de recherche infructueuse.

*Unbound super object* 
On s'intéresse au cas particulier de l'objet ```super``` retourné par un constructeur auquel on a passé qu'un seul argument. C'est un objet à l'usage restreint et peu évident et finalement déconseillé, l'*unbound super object* risquant même d'être déprécié dans le futur. 

L'*attribute access* via ```super``` réalise en fait la liaison ```__self__.some_method``` par exemple. Or dans le cas de l'*unbound super object*, cela n'est pas possible (```None.some_method``` n'ayant aucun sens et finirait inévitablement par une erreur). Il n'est pas possible d'utiliser un *unbound super object* directement pour l'*attribute access*, il faut nécessairement le *rebound* avant. Il est nécessaire pour cela d'utiliser la méthode ```__get__``` de ```super```. Ainsi :
```super(C).__get__(c)``` est équivalent à ```super(C, c)```
```super(C).__get__(C)``` est équivalent à ```super(C, C)```

Dit autrement : 
```super(C).some_method()``` va renvoyer une erreur (Python ne sait pas à quel objet *bound* l'objet retourné par ```__dict__['some_method']```). A l'opposé, ```super(C).__get__(c).some_method()``` fonctionne et est équivalent à ```super(C, c).some_method()```, on a donc aucun intérêt à utiliser la première syntaxe qui est en plus beaucoup plus obscure. 

Les restrictions s'appliquant à l'argument passé à ```__get__``` sont les mêmes que s'il avait été passé en seconde position au constructeur (```__get__``` retourne en fait un nouvel objet ```super```).

Il peut paraître étrange d'en passer par ```__get__``` pour *rebound* l'objet, mais cette méthode a justement été prévue pour le cas de l'*unbound super object* pour son seul réel cas d'usage : on place un *unbound super object* en attribut d'une classe, le fait qu'il soit aussi un (*non-data*) *descriptor* permet que l'objet retourné en cas d'*attribute access* soit *rebound* (à l'objet sur lequel s'est fait d'*attribute access*) et utilisable. L'*unbound super object* n'est pas fait pour être utilisé directement.

On termine sur un exemple d'implémentation de ```super``` en pur Python donnée par Guido van Rossum dans ce [post](https://www.python.org/download/releases/2.2.3/descrintro/#cooperation): 

```python
class Super(object):
    def __init__(self, type, obj=None):
        self.__type__ = type
        self.__obj__ = obj
    
    def __get__(self, obj, type=None):
        if self.__obj__ is None and obj is not None:
            return Super(self.__type__, obj)
        else:
            return self

    def __getattribute__(self, attr):
        if isinstance(self.__obj__, self.__type__):
            starttype = self.__obj__.__class__
        else:
            starttype = self.__obj__
        
        mro = iter(starttype.__mro__)
        for cls in mro:
            if cls is self.__type__:
                break

        for cls in mro:
            if attr in cls.__dict__:
                x = cls.__dict__[attr]
                if hasattr(x, "__get__"):
                    x = x.__get__(self.__obj__)
                return x
        raise AttributeError(attr)
```
## Autre
https://docs.python.org/3/reference/datamodel.html#implementing-descriptors
https://docs.python.org/3.7/howto/descriptor.html
https://docs.python.org/3.6/tutorial/classes.html#method-objects

https://late.am/post/2015/05/07/optimize-python-with-closures.html

Autre exemple de protocol : iterator protocol 
https://docs.python.org/2/library/stdtypes.html#iterator-types
https://docs.python.org/3/reference/compound_stmts.html#the-for-statement

Name binding / scoping
Binding is intimately connected with scoping, as scope determines which names bind to which objects – at which locations in the program code (lexically) and in which one of the possible execution paths (temporally). 

Sur __new__ https://www.python.org/download/releases/2.2.3/descrintro/#__new__

Attention aux attributs de classe : on rappelle qu'ils sont partagés par toutes les instances. Il est déconseillé qu'ils soient mutables.

Methods may reference global names in the same way as ordinary functions. The global scope associated with a method is the module containing its definition. (A class is never used as a global scope.) While one rarely encounters a good reason for using global data in a method, there are many legitimate uses of the global scope: for one thing, functions and modules imported into the global scope can be used by methods, as well as functions and classes defined in it. Usually, the class containing the method is itself defined in this global scope, and in the next section we’ll find some good reasons why a method would want to reference its own class.

In [5]:
vars(super)

mappingproxy({'__repr__': <slot wrapper '__repr__' of 'super' objects>,
              '__getattribute__': <slot wrapper '__getattribute__' of 'super' objects>,
              '__get__': <slot wrapper '__get__' of 'super' objects>,
              '__init__': <slot wrapper '__init__' of 'super' objects>,
              '__new__': <function super.__new__(*args, **kwargs)>,
              '__thisclass__': <member '__thisclass__' of 'super' objects>,
              '__self__': <member '__self__' of 'super' objects>,
              '__self_class__': <member '__self_class__' of 'super' objects>,
              '__doc__': 'super() -> same as super(__class__, <first argument>)\nsuper(type) -> unbound super object\nsuper(type, obj) -> bound super object; requires isinstance(obj, type)\nsuper(type, type2) -> bound super object; requires issubclass(type2, type)\nTypical use to call a cooperative superclass method:\nclass C(B):\n    def meth(self, arg):\n        super().meth(arg)\nThis works for class met

In [6]:
help(super)

Help on class super in module builtins:

class super(object)
 |  super() -> same as super(__class__, <first argument>)
 |  super(type) -> unbound super object
 |  super(type, obj) -> bound super object; requires isinstance(obj, type)
 |  super(type, type2) -> bound super object; requires issubclass(type2, type)
 |  Typical use to call a cooperative superclass method:
 |  class C(B):
 |      def meth(self, arg):
 |          super().meth(arg)
 |  This works for class methods too:
 |  class C(B):
 |      @classmethod
 |      def cmeth(cls, arg):
 |          super().cmeth(arg)
 |  
 |  Methods defined here:
 |  
 |  __get__(self, instance, owner, /)
 |      Return an attribute of instance, which is of type owner.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  --------------------------

In [12]:
class A(object):
    a = 1
    b = 2
    
class B(A):
    b = 3
    def __init__(self):
        self.hello = 1
        
    
    def hello(self):
        print('hello')

a=A()
b=B()

In [13]:
b.hello

1

In [21]:
super(B, b).c

AttributeError: 'super' object has no attribute 'c'

In [5]:
class SomeClass(object):
    def __init__(self):
        self._a = 0
        
    @property
    def a(self):
        print('Calling getter')
        return self._a
    
    @a.setter
    def a(self, value):
        print('Calling setter')
        self._a = value

b = SomeClass()
c = SomeClass()

In [9]:
b.a
c.a

Calling getter
Calling getter


0